# 🧪 Delta Lake Example Notebook

This notebook demonstrates key Delta Lake features:
- ACID transactions
- Schema enforcement and evolution
- Time travel
- Integration with Apache Spark


In [ ]:
# Create a Spark session with Delta support
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DeltaLakeDemo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [ ]:
# Create a DataFrame and save it as Delta
data = [(1, "Manzana", 10), (2, "Naranja", 15), (3, "Plátano", 7)]
columns = ["id", "fruta", "cantidad"]
df = spark.createDataFrame(data, columns)

delta_path = "/tmp/frutas_delta"
df.write.format("delta").mode("overwrite").save(delta_path)

In [ ]:
# Read and display the Delta table
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

In [ ]:
# Update data using DeltaTable API
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, delta_path)

delta_table.update(
    condition="fruta = 'Manzana'",
    set={"cantidad": "cantidad + 5"}
)

spark.read.format("delta").load(delta_path).show()

In [ ]:
# Time Travel - read previous version
df_old = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(delta_path)

df_old.show()